In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import pymongo
import json

# tool for browsing with Spliter
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser

In [2]:
# Initialize Pymongo to work with mongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

### Scrape for latest new title and text paragraph

In [3]:
news_url = 'https://mars.nasa.gov/news/'
response = requests.get(news_url)

# creat bs object
soup = bs(response.text, 'html.parser')

In [4]:
title_result = soup.find('div', class_='content_title')
news_title = title_result.text

In [5]:
teaser_result = soup.find('div', class_='rollover_description_inner')
teaser_para = teaser_result.text

### JPL Mars image--feature image

In [6]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/kaipham/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [7]:
# browse the link 
img_url = 'https://spaceimages-mars.com/'
browser.visit(img_url)

In [8]:
html = browser.html
soup = bs(html, 'html.parser')

# print(soup.prettify())

In [9]:
# click on the buttone "Full image"
# when it clicked the variable <html> also move to the new page, because the action if apply onto <browser>
browser.links.find_by_partial_text('FULL IMAGE').click()


In [10]:
html = browser.html
soup = bs(html, 'html.parser')

In [11]:
soup.find('img', class_='fancybox-image')['src']

'image/featured/mars1.jpg'

In [12]:
# Store the image url into a variable
feature_image_url = f"{img_url}{soup.find('img', class_='fancybox-image')['src']}"

### Mars Facts with Pandas Scraping (read_html)

In [13]:
fact_url = 'https://galaxyfacts-mars.com/'

In [14]:
tables = pd.read_html(fact_url)

In [15]:
mars_facts_df = tables[1]
mars_facts_df.columns = ["Entities", "Info"]

In [16]:
mars_earth_df = tables[0]
mars_earth_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [38]:
mars_earth_df.columns=['Mars - Earth Comparision','Mars','Earth']
mars_earth_df.drop(0, axis=0)

,Mars - Earth Comparision,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [41]:
# convert this table into json
mar_earth_json_df = mars_earth_df.to_json(orient="records")
parsed = json.loads(mar_earth_json_df)
json.dumps(parsed)

'[{"Mars - Earth Comparision": "Mars - Earth Comparison", "Mars": "Mars", "Earth": "Earth"}, {"Mars - Earth Comparision": "Diameter:", "Mars": "6,779 km", "Earth": "12,742 km"}, {"Mars - Earth Comparision": "Mass:", "Mars": "6.39 \\u00d7 10^23 kg", "Earth": "5.97 \\u00d7 10^24 kg"}, {"Mars - Earth Comparision": "Moons:", "Mars": "2", "Earth": "1"}, {"Mars - Earth Comparision": "Distance from Sun:", "Mars": "227,943,824 km", "Earth": "149,598,262 km"}, {"Mars - Earth Comparision": "Length of Year:", "Mars": "687 Earth days", "Earth": "365.24 days"}, {"Mars - Earth Comparision": "Temperature:", "Mars": "-87 to -5 \\u00b0C", "Earth": "-88 to 58\\u00b0C"}]'

### Mars Hamispheres

In [ ]:
hami_url = 'https://marshemispheres.com/'

In [ ]:
browser.visit(hami_url)

In [ ]:
# save the list of links that have the word "Enhanced" into the list
links_found = browser.links.find_by_partial_text('Enhanced')

In [ ]:
type(links_found)

### Teeeeesting out the step-by-step to find out algorithm

In [ ]:
# print(links_found[2])

In [ ]:
# click on the link #2
# links_found[].click()

In [ ]:
# click on the word "Sample"
# browser.links.find_by_text('Sample').click()

In [ ]:
# Bring the Browser session into the second tab, 
# so we can start scraping on that page (the page that contain JPG file)
# browser.windows.current = browser.windows[1]

In [ ]:
# # create soup object to start scraping
# html = browser.html
# soup = bs(html, 'html.parser')

In [ ]:
# # finding the the actual tag that contain the image url
# soup.find('img')['src']

In [ ]:
# # bring the window back to the first page and close all other window
# window = browser.windows[0]
# window.close_others()

In [ ]:
# bring the Browser session back to first window
# browser.windows.current = browser.windows[0]

### Finissssssssshhh testing

In [ ]:
image_urls = []

In [ ]:
for i in range(4): 
#     click on the link with associate order of i
    links_found[i].click()
    
#     find the word 'Enhanced' and click on the link
    browser.links.find_by_text('Sample').click()
    
#     the click on the link pop-up another window
#     Therefore, need to bring the current session to second window
    browser.windows.current = browser.windows[1]
    
#     reset html variable and soup variable, cuz at the end the whole found_links list is changed
    html = browser.html
    soup = bs(html, 'html.parser')
    
#     save the url into image variable
    image = soup.find('img')['src']
#     save the link into list/dictioary as needed
    image_urls.append(image)
    
    
#     back to the first window
    window = browser.windows[0]
#     close all other window
    window.close_others()
#     bring the browser session back to first window
    browser.windows.current = browser.windows[0]
    
#     get back to the main page with hami_url on the first window
    browser.visit(hami_url)
#     find all the links with 'Enhanced' again
    links_found = browser.links.find_by_partial_text('Enhanced')

In [ ]:
# quit Chrome, safe energy, ram and maintaining good practice
browser.quit()

In [ ]:
image_urls

In [ ]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere"},
    {"title": "Cerberus Hemisphere"},
    {"title": "Schiaparelli Hemisphere"},
    {"title": "Syrtis Major Hemisphere"}]

In [ ]:
pointer = 0
for title_dict in hemisphere_image_urls:
    title_dict.update({'img_url': image_urls[pointer]})
    pointer += 1

In [ ]:
hemisphere_image_urls